In [2]:
# Constants
BV_ss = 4.3
Ht_ss = .42
MCV_ss = 90*10**(-15)

BV_percent_loss = 0.2


In [3]:
Fe_molar = 55.845
Tf_molar = 79_000

n3 = 3.22*10**(-5)
n4 = 2.69*10**(-5)
n5 = 2.65*10**(-6)
n6 = n5

print(f'{n4*Fe_molar + n6*(Tf_molar+2*Fe_molar) - n3*Fe_molar - n5*Tf_molar:.15f}')
print(n4*Fe_molar*1000)
print(n6*(Tf_molar+2*Fe_molar)*1000)
print(n3*Fe_molar*1000)
print(n5*Tf_molar*1000)

print(1.5+209.6-209.3-1.8)

0.000000000000000
1.5022305000000002
209.64597849999996
1.798209
209.35
-1.709743457922741e-14


In [4]:
Hg_count = 270 * 10 ** 6 # Literature
Hg_molar = 65_000 # g/mol, Da, Literature (Journal of Biological Chemistry)
heme_molar = 616.487 # g/mol, Literature 

heme_count = Hg_count * 4 # molecules heme per RBC
print(heme_count)

1080000000


In [5]:
Avogadro_num = 6.022*10**23 # molecules / mol
heme_flow_rate = 3.6*10**(-4)
conversion_factor = 1/(heme_count/Avogadro_num) # mol heme per RBC

print(heme_flow_rate*conversion_factor/Avogadro_num)

heme_moles = heme_count/Avogadro_num # heme moles / RBC
g_RBC = heme_moles*heme_molar # g/RBC

200*10**9*g_RBC*1000
175*10**9*g_RBC*1000


3.333333333333334e-13


193.4839637994022

In [11]:
4.3*.58

2.4939999999999998

In [6]:
1000*Fe_molar*3.6*10**-4 # mol Fe * g/mol

20.104200000000002

### RBC count change

In [10]:
BV_ss = 4.3
Ht_ss = .42
MCV = 90*10**(-15)
print(f'{BV_ss*Ht_ss/MCV:,} RBCs')

BV_temp = BV_ss*(1-BV_percent_loss)
RBC_new = BV_temp*Ht_ss/MCV
Ht_new = RBC_new/BV_ss
print(BV_temp*(1-Ht_new))

20,066,666,666,666.664 RBCs
-12842666666663.225


In [11]:
(18*10**12*MCV)/BV_ss

0.37674418604651166

### RBC count change
$\dot{RBC} = \dot{Add} + \dot{Rem} $

### Adding Function
Assume: Stepwise increase from v_ss to v_max,ds 
- Interpolate between hematocrit change [33.6%, 42.0%] which reflects RBC change [16 trillion, 20 trillion]




In [ ]:
v_ss = 0.094 # µM/min
v_max_ds = 



### Removing Function
